In [ ]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = "retina"

In [ ]:
import addict
import pickle
import numpy as np
import scipy.sparse
import scipy.sparse.linalg
import matplotlib.pyplot as plt
from importlib import reload

import celeri
from celeri.hmatrix import build_hmatrix_from_mesh_tdes
from celeri.hmatrix import build_hmatrix_from_mesh_tdes_new

with open('hmatrix_dev_data.pkl', 'rb') as f:
    estimation, operators, meshes, segment, station, command, index = pickle.load(f)

In [ ]:
command_file_name = "../data/command/western_north_america_command.json"
command, segment, block, meshes, station, mogi, sar = celeri.read_data(command_file_name)
celeri.create_output_folder(command)
station = celeri.process_station(station, command)
segment = celeri.process_segment(segment, command, meshes)
sar = celeri.process_sar(sar, command)
closure, block = celeri.assign_block_labels(segment, station, block, mogi, sar)
assembly = addict.Dict()
operators = addict.Dict()
operators.meshes = [addict.Dict()] * len(meshes)
assembly = celeri.merge_geodetic_data(assembly, station, sar) # Not sure this works correctly

In [ ]:
# Get all elastic operators for segments and TDEs
celeri.get_elastic_operators(operators, meshes, segment, station, command)

# Get TDE smoothing operators
celeri.get_all_mesh_smoothing_matrices(meshes, operators)
celeri.get_all_mesh_smoothing_matrices_simple(meshes, operators)

operators.rotation_to_velocities = celeri.get_rotation_to_velocities_partials(station)
operators.global_float_block_rotation = celeri.get_global_float_block_rotation_partials(station)
assembly, operators.block_motion_constraints = celeri.get_block_motion_constraints(assembly, block, command)
assembly, operators.slip_rate_constraints = celeri.get_slip_rate_constraints(assembly, segment, block, command)
operators.rotation_to_slip_rate = celeri.get_rotation_to_slip_rate_partials(segment, block)
operators.block_strain_rate_to_velocities, strain_rate_block_index = celeri.get_block_strain_rate_to_velocities_partials(block, station, segment)
operators.mogi_to_velocities = celeri.get_mogi_to_velocities_partials(mogi, station, command)
celeri.get_tde_slip_rate_constraints(meshes, operators)
index2, estimation = celeri.assemble_and_solve_dense(command, assembly, operators, station, block, meshes)
celeri.post_process_estimation(estimation, operators, station, index2)


## Build H-matrix for Cascadia mesh

In [ ]:
H = build_hmatrix_from_mesh_tdes_new(
    meshes[0], 
    station,
    operators.tde_to_velocities[0],
    1e-6,
    min_separation=1.25,
    min_pts_per_box=20,
)
print(f"H matrix compression ratio: {H.report_compression_ratio():0.4}")

# Weighting

In [ ]:
W = estimation.weighting_vector
X = estimation.operator
y = estimation.data_vector
Xp = X * np.sqrt(W)[:, None]
yp = y * np.sqrt(W)

# Preconditioning
Scale each column of the data matrix to have a L2 norm of 1.

In [ ]:
col_norms = np.linalg.norm(Xp, axis=0)
XpP = Xp / col_norms[None, :]

## Combining H-matrices and iterative solvers

In [ ]:
from scipy.sparse import csr_matrix
operators.rotation_to_slip_rate_to_okada_to_velocities = operators.slip_rate_to_okada_to_velocities @ operators.rotation_to_slip_rate
sparse_block_motion_okada_faults = csr_matrix(operators.rotation_to_velocities[index.station_row_keep_index, :] - operators.rotation_to_slip_rate_to_okada_to_velocities[index.station_row_keep_index, :])
sparse_block_motion_constraints = csr_matrix(operators.block_motion_constraints)
sparse_block_slip_rate_constraints = csr_matrix(operators.slip_rate_constraints)
tde_keep_row_index = celeri.get_keep_index_12(operators.tde_to_velocities[0].shape[0])
tde_keep_col_index = celeri.get_keep_index_12(operators.tde_to_velocities[0].shape[1])
smoothing_keep_index = celeri.get_keep_index_12(operators.smoothing_matrix[0].shape[0])
tde_matrix = operators.tde_to_velocities[0][tde_keep_row_index, :][:, tde_keep_col_index]
sparse_tde_smoothing = csr_matrix(operators.smoothing_matrix[0][smoothing_keep_index, :][:, smoothing_keep_index])
sparse_tde_slip_rate_constraints = csr_matrix(operators.tde_slip_rate_constraints[0])

In [ ]:
def matvec(v):
    """ BJM: Build matvec (matrix vector product) operator for 
    scipy.sparse.linalg.LinearOperator.  This returns A* u

    BJM: Should we be passing in: W, X, index, etc. or let them be known from the outer scope???
    TBT: This will depend on how we integrate this into celeri and which
    variable we're talking about. For example, we should stop using X.shape
    entirely because that matrix won't exist in a fully sparse/hmatrix
    implementation!
    One design that I would probably lean towards
    would be something like:
    def build_sparse_hmatrix_linear_operator(operators,...):
        sparse_block_motion_okada_faults = ...
        define_other_precomputable_vars_here = ...

        def matvec(v):
            # use vars from the outer scope
        def rmatvec(v):
            # use vars from the outer scope

        return scipy.sparse.linalg.LinearOperator(X.shape, matvec=matvec, rmatvec=rmatvec)
        


    Args:
        u (nd.array): Candidate state vector

    Returns:
        out (nd.array): Predicted data vector
    """

    # BJM: Weight the data vector
    # TBT: It's important to remember to keep the input and output weighting
    # conceptually separate since the "out * np.sqrt(W)" will actually change
    # the solution to the least squares problem whereas the "v / col_norms"
    # preconditioning step is a reversible change to the solution (which is the
    # point since preconditioning should not change the solution!!)
    v_scaled = v / col_norms 

    # BJM: Make storage for output
    out = np.zeros(X.shape[0])

    block_rotations = v_scaled[index.start_block_col : index.end_block_col]
    # okada
    out[
        index.start_station_row : index.end_station_row
    ] += sparse_block_motion_okada_faults.dot(block_rotations)

    # block motion constraints
    out[
        index.start_block_constraints_row : index.end_block_constraints_row
    ] += sparse_block_motion_constraints.dot(block_rotations)

    # slip rate constraints
    out[
        index.start_slip_rate_constraints_row : index.end_slip_rate_constraints_row
    ] += sparse_block_slip_rate_constraints.dot(block_rotations)

    tde_velocities = v_scaled[index.meshes[0].start_tde_col : index.meshes[0].end_tde_col]

    # Insert TDE to velocity matrix
    out[index.start_station_row : index.end_station_row] += H.dot(tde_velocities)

    # TDE smoothing
    out[
        index.meshes[0].start_tde_smoothing_row : index.meshes[0].end_tde_smoothing_row
    ] += sparse_tde_smoothing.dot(tde_velocities)

    # TDE slip rate constraints
    out[
        index.meshes[0].start_tde_constraint_row :
        index.meshes[0].end_tde_constraint_row
    ] += sparse_tde_slip_rate_constraints.dot(tde_velocities)

    # Weight!
    return out * np.sqrt(W)

In [ ]:
def rmatvec(u):
    """ 
    Args:
        u (nd.array): Candidate state vector

    Returns:
        out (nd.array): Predicted data vector
    """

    # BJM: Weight the data vector
    u_weighted = u * np.sqrt(W)

    # BJM: Storage for output
    out = np.zeros(X.shape[1])

    # BJM: Select subset of weighted data for the observed velocities
    station_rows = u_weighted[index.start_station_row : index.end_station_row]
    block_constraints = u_weighted[
        index.start_block_constraints_row : index.end_block_constraints_row
    ]

    # BJM: Select subset of weighted data for the fault slip rate constraints
    slip_rate_constraints = u_weighted[
        index.start_slip_rate_constraints_row : index.end_slip_rate_constraints_row
    ]

    # BJM: Select subset of weighted data for the TDE smoothing
    tde_smoothing = u_weighted[
        index.meshes[0].start_tde_smoothing_row : index.meshes[0].end_tde_smoothing_row
    ]

    # BJM: Select subset of weighted data for the TDE slip rate constraints
    tde_slip_rate = u_weighted[
        index.meshes[0].start_tde_constraint_row :
        index.meshes[0].end_tde_constraint_row
    ]

    # BJM: Okada and block rotation contribution to data vector
    out[index.start_block_col : index.end_block_col] += station_rows @ sparse_block_motion_okada_faults

    # BJM: Block motion constraints contribution to data vector
    out[index.start_block_col : index.end_block_col] += block_constraints @ sparse_block_motion_constraints

    # BJM: Fault slip rate constraints contribution to data vector
    out[index.start_block_col : index.end_block_col] += slip_rate_constraints @ sparse_block_slip_rate_constraints

    # BJM: Hmatrix (TDEs to velocities)
    out[index.meshes[0].start_tde_col : index.meshes[0].end_tde_col] += H.transpose_dot(station_rows)

    # BJM: TDE smoothing contribution to data vector
    out[index.meshes[0].start_tde_col : index.meshes[0].end_tde_col] += tde_smoothing @ sparse_tde_smoothing

    # BJM: TDE slip rate constraint contributions to data vector
    out[index.meshes[0].start_tde_col : index.meshes[0].end_tde_col] += tde_slip_rate @ sparse_tde_slip_rate_constraints

    # Weight!
    return out / col_norms

## Solve with H-matrix/sparse!

In [ ]:
# BJM: Create a linear operator the includes the hmatrix vector multiply
op = scipy.sparse.linalg.LinearOperator(X.shape, matvec=matvec, rmatvec=rmatvec)

In [ ]:
lsmr_sparse = scipy.sparse.linalg.lsmr(op, yp, atol=1e-6, btol=1e-6)
lsmr_sparse_soln = lsmr_sparse[0] / col_norms
estimation.state_vector = lsmr_sparse_soln

In [ ]:
celeri.plot_estimation_summary(segment, station, meshes, estimation, lon_range=(225, 250), lat_range=(30, 52), quiver_scale=1e2)